<a href="https://colab.research.google.com/github/behnaznikkhah/Project/blob/master/ImpactofCovidonStockmarket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [5]:
import os
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from numpy import concatenate
from math import sqrt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

#reading files
file_path = os.path.abspath("")
data_pandemic = pd.read_csv(os.path.join(file_path, 'PandemicData.csv'))#, parse_dates=['Date'], index_col='Date')
data_index = pd.read_csv(os.path.join(file_path, 'Data.csv'))#, parse_dates=['Date'], index_col='Date')
#cumalative cases and deaths
pandemicd = data_pandemic
pandemicd=pandemicd.groupby(['Date','Pandemic'],as_index=False).sum()
pandemicd=pandemicd[pandemicd.Pandemic.eq("Covid")]
pandemicd = pandemicd.drop(['Pandemic'],1)
#stock market data
marketd = data_index
countrynames = marketd.Country.unique()
country = input("Enter country: ")
if country in  countrynames:
    marketd=marketd[marketd.Country.eq(country)]
else:
    print("List of countries are:",countrynames)
marketd = marketd.drop(['Volume','Low','Open','Change','High','Country','Vol.'], 1)
#join two data
result = pd.merge(pandemicd, marketd, how='inner', on=['Date'])
#change the order of columns
result=result[['Date','Price','ActiveCases','Deaths']]

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

# load dataset
dataset = result
dataset=dataset.set_index('Date')
#dataset.index = pd.to_datetime(dataset['Date'])
values = dataset.values
# integer encode direction
#encoder = LabelEncoder()
#values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 30,1)
print(len(reframed.columns))
#print(reframed.head)


# drop columns we don't want to predict
#reframed.iloc[:,lambda reframed: reframed.index % 3 == 0]
lst = [x for x in range(0, 90) if x % 3 == 0]
reframed.drop(reframed.columns[lst], axis=1, inplace=True)
reframed.drop(reframed.columns[[61,62]], axis=1, inplace=True)

#print(reframed.columns)
#print(reframed.head)

# split into train and test sets
values = reframed.values
#print((values))


n_train = 150
train = values[:n_train, :]
test = values[n_train:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
from keras.layers import LeakyReLU

tf.keras.backend.clear_session()
tf.random.set_seed(51)
np.random.seed(51)
#,activation='relu'
tf.keras.backend.clear_session()
model = Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(LeakyReLU())
model.add(LSTM(10))
model.add(LeakyReLU())
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam',metrics=["mae"])
# fit network
history=model.fit(train_X, train_y, epochs=50,batch_size=1, validation_data=(test_X, test_y), verbose=0)

"""
# Evaluate your model 
print("Final loss value:",model.evaluate(test_X, test_y))

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()
"""
# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = numpy.array(new_row)
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

forecast = []
for time in range(len(values)):
	X=values[time:time+1][:,:-1]
	X = X.reshape((1, 1, 60))
	yhat = model.predict(X)
	# invert scaling
	yhat = invert_scale(scaler, X, yhat)
	forecast.append(yhat)
results = np.array(forecast)[:, 0, 0]


plt.plot(results,label='forecast')
plt.plot(values[:,:][:,-1],label='actual')
plt.legend()
plt.show()


print(tf.keras.metrics.mean_absolute_error(values[:,-1], results).numpy())
# calculate RMSE
rmse = sqrt(mean_squared_error(values[:,-1], results))
print('Test RMSE: %.3f' % rmse)


"""



# make a prediction
yhat = model.predict(test_X)
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]


# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
"""
"""
yhat = model.predict(test_X)
# calculate RMSE
rmse = sqrt(mean_squared_error(test_y, yhat))
print('Test RMSE: %.3f' % rmse)"""

93
(150, 1, 60) (150,) (30, 1, 60) (30,)


KeyboardInterrupt: ignored

In [ ]:
from numpy import array
x_input = array([[80, 85], [90, 95], [100, 105]])
print(x_input[0:2][:,:-1])
x_input = x_input.reshape((1, 3, 2))
print(x_input)

[[80]
 [90]]
[[[ 80  85]
  [ 90  95]
  [100 105]]]
